# How to build simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App
    * Will be able to have conversation
    * Will remember previous interactions: will have memory
    * Will be able to store memory in json file


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages
    *  LLM Model is based around raw text
* Chat History: allows Chat Model to remember previous interactions

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!pip list | grep langchain

langchain                 0.2.1
langchain-community       0.2.1
langchain-core            0.2.2
langchain-openai          0.1.8
langchain-text-splitters  0.2.0

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Connect with LLM and start conversation with it

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

### Call ChatModel (LLM)

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='Blue is a calming and peaceful color that represents tranquility and serenity. It is often associated with the sky and the ocean, evoking feelings of freedom and openness. Blue is also known to symbolize trust, loyalty, and wisdom. Overall, blue is a versatile and timeless color that is loved by many.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 13, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c04030ec-cc97-4525-bcb6-617d9b705b64-0', usage_metadata={'input_tokens': 13, 'output_tokens': 63, 'total_tokens': 76})

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Check if Chatbot remembers our favorite color

In [9]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have access to that information.", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a31d0a71-a438-4cf2-aae1-8b992b0581a7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26})

## Add memory to Chatbot

In [11]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [12]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [13]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [14]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv053024-p2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [15]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [16]:
chain.invoke("my name is Julio")

{'content': 'my name is Julio',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Nice to meet you, Julio! How can I help you today?'}

In [17]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is Julio'),
  AIMessage(content='Nice to meet you, Julio! How can I help you today?')],
 'text': 'Your name is Julio.'}

### Check file messages.json in root directory